# HW04

## 1. Преобразуем таблицу с абсолютными частотностями в таблицу с tfidf значениями.

In [1]:
from IPython.display import Image
from IPython.core.display import HTML

IMG_URL = 'https://i.ibb.co/47bRcVy/bow-normalized.jpg'
Image(url=IMG_URL, width=500, height=500)

In [2]:
IMG_URL = 'https://raw.githubusercontent.com/safonovanastya/hse_comp_ling/main/4/Screenshot.png'
Image(url=IMG_URL, width=500, height=500)

## 2а) Посчитаем близость между 3 и 12666 текстами в датасете (labeled.csv из семинара) 

In [3]:
import string
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords

In [4]:
df = pd.read_csv('data/labeled.csv')
df.comment = df.comment
df.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [5]:
vect = TfidfVectorizer()
data = vect.fit_transform(df.comment) 

In [6]:
print('близость: ' + str(cosine_similarity(data[3], data[12666][0][0])[0][0]))

близость: 0.27330886297549256


## 2б) Найдём 3 самых близких текста к тексту номер 43

In [7]:
closest = cosine_distances(data[43], data).argsort()[0,1:4]
proximity = np.sort(cosine_distances(data[43], data))[0,1:4]
print(closest, proximity)

[1986 1957 6259] [0.83956018 0.88210851 0.88590919]


In [8]:
print(closest)

[1986 1957 6259]


In [9]:
for ind, n in enumerate(closest):
    print('текст: ' + df.loc[n].comment)
    print('близость: ' + str(1 - proximity[ind]))

текст: НУ И КАКАЯ МРАЗЬ КИДАЕТ ССЫЛКИ? ОХУЕЛИ ТАМ В КРАЙ УЖЕ?

близость: 0.16043982092396114
текст: Че за бригада и че за махоун? Из полицейской академии?
близость: 0.11789148760536672
текст: Герка ебет только даунов которые игрли а него. Ибо и геймплей и сюжетто кусок говна.

близость: 0.11409080923099102


## 3. Обучим 2 любых классификатора

In [10]:
stops = stopwords.words('russian')
X_train, X_test, y_train, y_test = train_test_split(df.comment, df.toxic, test_size=0.1, shuffle=True, random_state=23)

### 1 -- Логистическая регрессия

In [11]:
vect_count = CountVectorizer(ngram_range=(1,2), max_df=0.85, min_df=2,
                             stop_words=stops, analyzer='char_wb')
X_train_count = vect_count.fit_transform(X_train)
X_test_count = vect_count.transform(X_test)

In [12]:
lr = LogisticRegression(C=0.1, class_weight='balanced')
lr.fit(X_train_count, y_train)

preds = lr.predict(X_test_count)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.90      0.82      0.86       931
         1.0       0.71      0.83      0.77       511

    accuracy                           0.82      1442
   macro avg       0.81      0.82      0.81      1442
weighted avg       0.83      0.82      0.82      1442



/home/nastia/miniconda3/envs/python3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 2 -- Дерево решений

In [13]:
vect_tfidf = TfidfVectorizer(ngram_range=(1,2), min_df = 3, max_df = 0.95, max_features = 1000, token_pattern=r'(?u)\b\w+\b')

X_train_vect = vect_tfidf.fit_transform(X_train)
X_test_vect = vect_tfidf.transform(X_test)

In [14]:
dec_tree = DecisionTreeClassifier(criterion='entropy', max_depth=30, random_state=127, min_samples_split=2)
dec_tree.fit(X_train_vect, y_train)

preds = dec_tree.predict(X_test_vect)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.76      0.79      0.78       931
         1.0       0.59      0.55      0.57       511

    accuracy                           0.71      1442
   macro avg       0.68      0.67      0.67      1442
weighted avg       0.70      0.71      0.70      1442



### Работа с корпусом Dvach

In [15]:
dvach = open('data/2ch_corpus.txt', encoding="utf-8").readlines()[0:2500]

In [16]:
dvach_count = vect_count.transform(dvach)
dvach_vect = vect_tfidf.transform(dvach)

In [21]:
dvach_lr_proba = lr.predict_proba(dvach_count)

dvach_dt_proba = dec_tree.predict_proba(dvach_vect)

In [24]:
lr_top10 = dvach_lr_proba[:,1].argsort()[::-1][:10]
lr_top10_proba = dvach_lr_proba[lr_top10, 1]

dt_top10 = dvach_dt_proba[:,1].argsort()[::-1][:10]
dt_top10_proba = dvach_dt_proba[dt_top10, 1]

In [42]:
for ind, i in enumerate(lr_top10):
    print(dvach[i])
    print('токсик на ' + str(lr_top10_proba[ind]*100) + '%')
    print('===========================================================================')

  ВИЧУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУЧУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУЧУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУЧУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУУХА БЛЯЯЯЯЯЯЯЯ

токсик на 100.0%
 Ёбаные советские названия блять. КРУЖКИ блять. Всегда ненавидел. Блять. Почему бы не сказать КЛУБ ПО ИНТЕРЕСАМ, как у япошек, нет блять, будем гуманитарную хуйню, сравнения блять, типа В КРУГ СОБРАЛИСЬ, ахахахаха кружок ахахахахаха))))) пиздец блять. Может при Сталине это звучало, но уже при Горбачёве это просто бесило, а сейчас это выглядит вообще как атавизм. Или ещё ебанутое слово СЛЁТ блять. Сука, СЛЁТ, почему слёт, а не съезд? Типа съезд - это для КПСС, не доросли ещё? Или типа в Совке так всё пиздато, что даже у школия есть своё самолёт? СЛЁТ сука, мы что, блять, стрижи? Или это какая-то аллегория на ёбаных ОРЛЯТ? Вот тоже за

In [41]:
for ind, i in enumerate(dt_top10):
    print(dvach[i])
    print('токсик на ' + str(dt_top10_proba[ind]*100) + '%')
    print('===========================================================================')

 Анимублядский WebM-треддля приличных анимублядей и прочих аутистов. Безграмотное быдло с дубляжом, войсовером, порнографией и котиками, советы мерзких мокрописечников, вниманиебляди всех видов и прочее непотребство отправляется в порнотред <ссылка>.Для поиска сoуса видео сохраняем кадр (правый клик по видео) и ищем его на Для воспроизведения WebM с 10-битным цветом нужно установить плагин vlc ( ) и отключить встроенный в браузер плеер (media. webm. enabled=false в firefox).О кодировании WebMДоступные кодеки — VP8 и VP9 для видео, Vorbis и Opus для звука, максимальный размер файла — 10240КБ, всех файлов в посте — около 40МБ. Делать WebM можно научиться в вики треда: Там находится подробная информация о выборе и настройке кодеков на примерах использования консольных утилит ffmpeg, vpxenc и mkvmerge. Неочевидные моменты— libvorbis при указании битрейта (-b:a) работает в режиме CBR (постоянный битрейт), и это портит качество звука; для режима VBR вместо битрейта надо указывать качество (-

Тексты не пересекаются. 

В логистической регрессии как токсичный отмечен немного странный комментарий "В каком там классе учили уравнения составлять? шахматисты + танцоры + шахматисты_танцоры = 56шахматисты + шахматисты_танцоры = 28танцоры + шахматисты_танцоры = 48танцоры - 28шахматисты_танцоры - 20шахматисты - 8" и самый первый, который он определил на 100% токсичный тоже немного странный, все остальные вплоне токсичны.

Дерево решений справилось чуть хуже, попадаются очевидно не токсичные комментарии (например, "Ты знаешь, а он в чём-то прав" или "Скорее всего эпигастральная фаза аппендицита, вызывай скорую говорю же, вангую что по анализам лейкоцитоз будет, если ты с дс приезжай я тебя прооперирую, как раз дежцрю седня)"). 